# Introduction

## Original Dataset Description
"This is a transnational data set which contains all the transactions occurring between 01/12/2010 and 09/12/2011 for a UK-based and registered non-store online retail.The company mainly sells unique all-occasion gifts. Many customers of the company are wholesalers."(https://www.kaggle.com/carrie1/ecommerce-data)

##### Business Question
Generate a time series forecast for the monthly/weekly sales.

## Potential Challenges
-  Only one-year data: seasonality matters
-  Stationarity: Most of the time series models work on the assumption that the time series is stationary

# Data Preprocessing

## Dataset overview

In [391]:
# import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('mode.chained_assignment', None)

In [392]:
# read data
df_raw=pd.read_csv('data.csv',encoding="ISO-8859-1")
# show first and last 5 rows of data
df_raw.head()
df_raw.tail()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/9/2011 12:50,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/2011 12:50,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/2011 12:50,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/2011 12:50,4.15,12680.0,France
541908,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,12/9/2011 12:50,4.95,12680.0,France


In [393]:
# data types and basic statistics
df_raw.dtypes
df_raw.describe()

InvoiceNo       object
StockCode       object
Description     object
Quantity         int64
InvoiceDate     object
UnitPrice      float64
CustomerID     float64
Country         object
dtype: object

,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


## Data Preparation
This dataframe contains 8 variables: InvoiceNo, StockCode, Description, Quantity, InvoiceDate, UnitPrice, CustomerID, and Country. <br>
**InvoiceNo**: Invoice number. Nominal, Uniquely and chronologically assigned to each transaction.  <br>
**StockCode**: Product (item) code. Uniquely assigned to each distinct product. <br>
**Description**: Product (item) name. Nominal. <br>
**Quantity**: The quantities of each product (item) per transaction. Numeric.	<br>
**InvoiceDate**: Invice Date and time. Numeric, the day and time when each transaction was generated. <br>
**UnitPrice**: Unit price. Numeric, Product price per unit in sterling. <br>
**CustomerID**: Customer number. Nominal, a 5-digit integral number uniquely assigned to each customer. <br>
**Country**: Country name. Nominal, the name of the country where each customer resides.<br>

### Drop duplications

In [394]:
print('Number of duplicated transactions: ',df_raw.duplicated().sum())
df=df_raw.drop_duplicates()

Number of duplicated transactions:  5268


### Missing values

In [395]:
# fill out missing customerID with 0
df['CustomerID']=df['CustomerID'].fillna(0)

### InvoiceDate
In order to apply a time series analysis, the data type of **InvoiceDate** is converted to *datetime*.

In [396]:
df['InvoiceDate']=pd.to_datetime(df['InvoiceDate'])

### Special transactions

A portion of transactions have negative quantities and zero unitprices, which requires extra attentions. 

In [397]:
print('Number of transactions that have negative quantities:', len(df[df['Quantity']<0]))
print('Number of transactions that have zero unitprices:', len(df[df['UnitPrice']==0]))

Number of transactions that have negative quantities: 10587
Number of transactions that have zero unitprices: 2510


Also, some of these orders have speical letters in their InoiceNo.

In [398]:
df['InvoiceNo'].str[0].value_counts()

5    527387
C      9251
A         3
Name: InvoiceNo, dtype: int64

We can see there are 9254 (9251+3) transactions' InoviceNo start with a letter, either "C" or "A". 2510 transactions' unitPrice are zero. Let's take a closer look at these transactions.

In [399]:
df[df['InvoiceNo'].str[0]=='C'].head()
df[df['InvoiceNo'].str[0]=='A'].head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
141,C536379,D,Discount,-1,2010-12-01 09:41:00,27.50,14527.0,United Kingdom
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,2010-12-01 09:49:00,4.65,15311.0,United Kingdom
235,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,2010-12-01 10:24:00,1.65,17548.0,United Kingdom
236,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom
237,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
299982,A563185,B,Adjust bad debt,1,2011-08-12 14:50:00,11062.06,0.0,United Kingdom
299983,A563186,B,Adjust bad debt,1,2011-08-12 14:51:00,-11062.06,0.0,United Kingdom
299984,A563187,B,Adjust bad debt,1,2011-08-12 14:52:00,-11062.06,0.0,United Kingdom


Appearently, if this code starts with letter 'C', it indicates a cancellation. If this code starts with letter 'A', it indicates an adjust bad debt.

#### Adjust bad debt
Simply remove these three transactions.

In [400]:
df=df[df['InvoiceNo'].str[0]!='A']

#### Discount

Some of the cancellations are due to discount. 

In [401]:
df_discount=df[df['StockCode']=='D']
df_discount.head()
print('Number of discount transactions:', len(df_discount))

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
141,C536379,D,Discount,-1,2010-12-01 09:41:00,27.50,14527.0,United Kingdom
9038,C537164,D,Discount,-1,2010-12-05 13:21:00,29.29,14527.0,United Kingdom
14498,C537597,D,Discount,-1,2010-12-07 12:34:00,281.00,15498.0,United Kingdom
19392,C537857,D,Discount,-1,2010-12-08 16:00:00,267.12,17340.0,United Kingdom
31134,C538897,D,Discount,-1,2010-12-15 09:14:00,5.76,16422.0,United Kingdom


Number of discount transactions: 77


Without losing generality, these transactions can also be removed.

In [402]:
df=df[df['StockCode']!='D']

#### Cancellation

As for a cancellation, what will be executed here is to remove both the cancellation and its original order from the dataset. If an order is cancelled after being placed, it shouldn't be treated as a normal order. Here, the original order coresponds to a cancellation is determined by finding the nearest order that has the same **StockCode**, **Description**, **UnitPrice**, **CustomerID**, **Country** and **Quantity** (but positive). Note that some cancellations' original orders may not be included in this dataset, which probably because these orders were placed before 12/01/2010.

In [403]:
# extract all cancellations to a new dataframe df_cancel and remove these transactions from df
df_cancel=df[df['InvoiceNo'].str[0]=='C']
df=df[df['InvoiceNo'].str[0]!='C']
# remove the intial "C" letter for all the cancellations and change the quantity from negative to positive
df_cancel['InvoiceNo']=df_cancel['InvoiceNo'].str[1:]
df_cancel['Quantity']=-df_cancel['Quantity']
df_cancel.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
154,536383,35004C,SET OF 3 COLOURED FLYING DUCKS,1,2010-12-01 09:49:00,4.65,15311.0,United Kingdom
235,536391,22556,PLASTERS IN TIN CIRCUS PARADE,12,2010-12-01 10:24:00,1.65,17548.0,United Kingdom
236,536391,21984,PACK OF 12 PINK PAISLEY TISSUES,24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom
237,536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom
238,536391,21980,PACK OF 12 RED RETROSPOT TISSUES,24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom


Now the **InvoiceNo** column for both *df* and *df_cancel* only contain intergral numbers, so they can be converted to numerics.

In [404]:
df['InvoiceNo']=df['InvoiceNo'].astype('int32')
df_cancel['InvoiceNo']=df_cancel['InvoiceNo'].astype('int32')

Then, *df* and *df_cancel* are merged based on **StockCode**, **Description**, **UnitPrice**, **CustomerID**, **Country** and **Quantity** from *df_cancel* and drop the cancellations that cannot find original orders in *df*. Also, we only keep the orders that were placed before the corresponding cancellation.

In [405]:
# merge df and df_cancel and keep the original index of df to the new dataset
df_origin_canditates=df.reset_index().merge(df_cancel,how='right',
                              on=['StockCode', 'Description','Quantity','UnitPrice','CustomerID','Country']).set_index('index')
# remove those cancellations that cannot find exact same orders in the dataset
df_origin_canditates.dropna(axis = 0, subset = ['InvoiceNo_x'], inplace = True)
# only keep the orders that were placed before the cancellation
df_origin_canditates=df_origin_canditates.loc[lambda x: x['InvoiceNo_x']<x['InvoiceNo_y']]
df_origin_canditates.head()
print('number of all possible original orders:',len(df_origin_canditates))

,InvoiceNo_x,StockCode,Description,Quantity,InvoiceDate_x,UnitPrice,CustomerID,Country,InvoiceNo_y,InvoiceDate_y
index,,,,,,,,,,
55.0,536373.0,21071,VINTAGE BILLBOARD DRINK ME MUG,6,2010-12-01 09:02:00,1.06,17850.0,United Kingdom,543611,2011-02-10 14:38:00
72.0,536375.0,21071,VINTAGE BILLBOARD DRINK ME MUG,6,2010-12-01 09:32:00,1.06,17850.0,United Kingdom,543611,2011-02-10 14:38:00
285.0,536396.0,21071,VINTAGE BILLBOARD DRINK ME MUG,6,2010-12-01 10:51:00,1.06,17850.0,United Kingdom,543611,2011-02-10 14:38:00
422.0,536406.0,21071,VINTAGE BILLBOARD DRINK ME MUG,6,2010-12-01 11:33:00,1.06,17850.0,United Kingdom,543611,2011-02-10 14:38:00
3134.0,536602.0,21071,VINTAGE BILLBOARD DRINK ME MUG,6,2010-12-02 08:34:00,1.06,17850.0,United Kingdom,543611,2011-02-10 14:38:00


number of all possible original orders: 4138


Then we can select the latest order among the possible original orders for each cancellation.

In [406]:
df_origin = df_origin_canditates[df_origin_canditates.groupby(['InvoiceNo_y'])['InvoiceNo_x'].transform(max) == df_origin_canditates['InvoiceNo_x']]
df_origin.head()
print('Number of orginal orders: ', len(set(df_origin.index)))

,InvoiceNo_x,StockCode,Description,Quantity,InvoiceDate_x,UnitPrice,CustomerID,Country,InvoiceNo_y,InvoiceDate_y
index,,,,,,,,,,
4515.0,536787.0,21071,VINTAGE BILLBOARD DRINK ME MUG,6,2010-12-02 15:24:00,1.06,17850.0,United Kingdom,543611,2011-02-10 14:38:00
24199.0,538343.0,22114,HOT WATER BOTTLE TEA AND SYMPATHY,4,2010-12-10 14:29:00,3.95,13448.0,United Kingdom,538670,2010-12-13 16:19:00
109089.0,545582.0,21154,RED RETROSPOT OVEN GLOVE,200,2011-03-04 09:05:00,1.06,13694.0,United Kingdom,548577,2011-04-01 10:46:00
1312.0,536533.0,22680,FRENCH BLUE METAL DOOR SIGN 5,20,2010-12-01 13:31:00,1.25,16955.0,United Kingdom,538725,2010-12-14 11:16:00
42332.0,539984.0,C2,CARRIAGE,1,2010-12-23 14:58:00,50.00,14911.0,EIRE,540164,2011-01-05 12:02:00


Number of orginal orders:  2497


Next, we remove these orders from *df*.

In [407]:
df=df[~df.index.isin(set(df_origin.index))]

#### Return

There are still some transactions that have negative quantities and zero unit prices, which are corresponding to return orders. Here, an assumption is made that if an order is returned after being placed, it can still be treated as a normal order. Therefore, only the return transactions, not the original ones, are removed from the dataset.

In [408]:
df_return=df[(df['Quantity']<0)&(df['UnitPrice']==0)]
df_return.head()
df=df[~((df['Quantity']<0)&(df['UnitPrice']==0))]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
2406,536589,21777,NaN,-10,2010-12-01 16:50:00,0.0,0.0,United Kingdom
4347,536764,84952C,NaN,-38,2010-12-02 14:42:00,0.0,0.0,United Kingdom
7188,536996,22712,NaN,-20,2010-12-03 15:30:00,0.0,0.0,United Kingdom
7189,536997,22028,NaN,-20,2010-12-03 15:30:00,0.0,0.0,United Kingdom
7190,536998,85067,NaN,-6,2010-12-03 15:30:00,0.0,0.0,United Kingdom


#### Other adjustments

There are still some transactions have zero unitprice. 

In [409]:
df_other=df[df['UnitPrice']==0]
df_other.head()
list(df_other['Description'].value_counts().index)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
622,536414,22139,NaN,56,2010-12-01 11:52:00,0.0,0.0,United Kingdom
1970,536545,21134,NaN,1,2010-12-01 14:32:00,0.0,0.0,United Kingdom
1971,536546,22145,NaN,1,2010-12-01 14:33:00,0.0,0.0,United Kingdom
1972,536547,37509,NaN,1,2010-12-01 14:33:00,0.0,0.0,United Kingdom
1987,536549,85226A,NaN,1,2010-12-01 14:34:00,0.0,0.0,United Kingdom


['check',
 'found',
 'adjustment',
 'FRENCH BLUE METAL DOOR SIGN 1',
 'amazon',
 'Found',
 'FRENCH BLUE METAL DOOR SIGN 8',
 'Amazon',
 'RECIPE BOX PANTRY YELLOW DESIGN',
 'FRENCH BLUE METAL DOOR SIGN 4',
 'FRENCH BLUE METAL DOOR SIGN No',
 'FRENCH BLUE METAL DOOR SIGN 3',
 'OWL DOORSTOP',
 'FRENCH BLUE METAL DOOR SIGN 7',
 'FRENCH BLUE METAL DOOR SIGN 5',
 'Manual',
 '?',
 'FRENCH BLUE METAL DOOR SIGN 6',
 'RED KITCHEN SCALES',
 'had been put aside',
 'MINT KITCHEN SCALES',
 'BOX OF 24 COCKTAIL PARASOLS',
 'FRENCH BLUE METAL DOOR SIGN 9',
 'POLYESTER FILLER PAD 40x40cm',
 'POLYESTER FILLER PAD 45x45cm',
 'RECIPE BOX BLUE SKETCHBOOK DESIGN',
 'RED RETROSPOT CHARLOTTE BAG',
 'FRENCH BLUE METAL DOOR SIGN 2',
 'DOORMAT WELCOME TO OUR HOME',
 'IVORY KITCHEN SCALES',
 'CHILDS GARDEN SPADE BLUE',
 'FRENCH BLUE METAL DOOR SIGN 0',
 'GLASS JAR DIGESTIVE BISCUITS',
 'CHILDRENS GARDEN GLOVES BLUE',
 'BLACK KITCHEN SCALES',
 'GLASS JAR PEACOCK BATH SALTS',
 'ENAMEL WATERING CAN CREAM',
 'DOORMAT 

We can see these transactions are due to various reasons such as missing price, adjustment, check, found, etc. For simplicity, these transactions are droped as well. 

In [410]:
df=df[df['UnitPrice']!=0]

### Country

Check the data distribution across different countries.

In [411]:
# top 5 countries' percentages
countries = df['Country'].value_counts()
countries.apply(lambda x:x/len(df)*100).head()

United Kingdom    91.505226
Germany            1.715801
France             1.593859
EIRE               1.484934
Spain              0.469964
Name: Country, dtype: float64

We can see about 91% of the data are from UK. In the following, in order to achieve more accurate time series results, we only focus on the orders from UK.

In [412]:
df=df[df['Country']=='United Kingdom']

### Add "Sales" feature

A new column is created for characterizing the total sales of each transaction.

In [413]:
df['Sales']=df['Quantity']*df['UnitPrice']
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Sales
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34


## Data Visualization

The modified Data is exported to data_modified.csv in order to do data Vislization in Tableau.

In [414]:
df.to_csv('data_modified.csv', sep='\t')

## Data Analysis

## Discussion

## Summary